# 0. Préparation de l'environnement de travail

Pour le bon déroulé du projet, on installe les packages suivants : 
- requests : pour la récupération des données par API
- pandas : pour la création et la manipulation de tableaux de données
- zipfile : pour ouvrir, lire ou extraire des données compressées
- os : pour interagir avec le système de fichiers

In [2]:
import requests
import pandas as pd
import zipfile
import os

# 1. Importation des données

Des API étaient disponibles pour l'ensemble de nos jeux de données, mais nous avons préféré les télécharger en brut et les mettre en cache local sur S3 pour travailler à partir de là, afin de favoriser la reproductibilité du projet. 

## a. Infrastructures sportives et culturelles

### i. Données

Dans un premier temps, on récupère les données relatives aux infrastructures sportives et culturelles en France. L'Insee (sur **[insee.fr](https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_BPE_SPORT_CULTURE)**) propose une base d'équipements et de service accessibles à la population au niveau communal, intercommunal, départemental, régional... 

> Nous nous intéresserons au **niveau communal**, dans l'optique de faire de même pour les résultats aux élections présidentielles. 

Ce jeu de données est valide pour le **1er janvier 2024**, moins de deux ans après les élections présidentielles. L'écart de temps est donc faible étant donné le temps de construction et la durée de vie en général des infrastructures étudiées, d'où la pertinence de cette base de données. 

<span style="color:red">Le fichier csv téléchargé a été stocké sur notre espace S3 personnel, il faudra probablement changer le chemin d'accès pour y accéder. </span>

In [3]:
path_infrastructures_s3 = "s3://clemencelucas/projet_python_vote_rn/data/raw/DS_BPE_SPORT_CULTURE_2024_data.csv"
df_infrastructures = pd.read_csv(path_infrastructures_s3, sep=';') 

/tmp/ipykernel_62233/537846119.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infrastructures = pd.read_csv(path_infrastructures_s3, sep=';')


Pour vérifier que le data frame est bien chargé et pour avoir un premier aperçu des données, on fait quelques vérifications. 

In [4]:
df_infrastructures.head()

,GEO,GEO_OBJECT,FACILITY_DOM,FACILITY_SDOM,FACILITY_TYPE,INDOOR,LIGHTED,PRACTICE_AREA_ACCESSIBILITY,SANITARY_ACCESSIBILITY,LOCKER_ROOM_ACCESSIBILITY,FREE_ACCESS,SHOWER,SANITARY,SEASONAL_OPENING,MULTIPLEX_CINEMA,ERP_CATEGORY,BPE_MEASURE,TIME_PERIOD,OBS_VALUE
0,69,DEP,F,F1,F101,1,1,0,0,_U,_U,_U,1,0,_Z,2,FACILITIES,2024,1
1,12,DEP,F,F1,F101,0,0,0,0,1,_U,_U,1,1,_Z,3,FACILITIES,2024,1
2,59,DEP,F,F1,F101,1,1,0,0,0,0,1,1,0,_Z,1,FACILITIES,2024,1
3,18,DEP,F,F1,F101,1,1,1,0,0,0,1,1,1,_Z,3,FACILITIES,2024,1
4,45,DEP,F,F1,F101,1,1,1,1,1,_U,_U,1,0,_Z,3,FACILITIES,2024,1


In [5]:
df_infrastructures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351761 entries, 0 to 1351760
Data columns (total 19 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   GEO                          1351761 non-null  object
 1   GEO_OBJECT                   1351761 non-null  object
 2   FACILITY_DOM                 1351761 non-null  object
 3   FACILITY_SDOM                1351761 non-null  object
 4   FACILITY_TYPE                1351761 non-null  object
 5   INDOOR                       1351761 non-null  object
 6   LIGHTED                      1351761 non-null  object
 7   PRACTICE_AREA_ACCESSIBILITY  1351761 non-null  object
 8   SANITARY_ACCESSIBILITY       1351761 non-null  object
 9   LOCKER_ROOM_ACCESSIBILITY    1351761 non-null  object
 10  FREE_ACCESS                  1351761 non-null  object
 11  SHOWER                       1351761 non-null  object
 12  SANITARY                     1351761 non-null  object
 1

On remarque en premier lieu qu'il n'y a pas de cases vides car, pour chaque colonne, on a 1 351 761 non-null correspondant à l'entièreté des lignes.  

> **Remarque** : Aucune case n'est vide, mais certaines comportent un code signifiant "Indéterminé" (_U) ou "Sans objet" (_Z). On s'intéressera au problème des valeurs manquantes par la suite.

On observe également que panda interprète la quasi totalité des colonnes comme des types *object* donc on lui indique quels sont les types des colonnes qui nous intéressent. 
- GEO : elle comporte un code, une clé qui permet d'identifier une zone géographique. On pourrait penser qu'il s'agit forcément d'un entier, mais les départements Corses sont 2A et 2B et comportent une lettre, et il arrive que des communes aient un code commençant par 0. Il vaut donc mieux considérer cette colonne comme étant de type *string*
- GEO_OBJECT : elle indique le niveau géographique de la ligne (COM pour commune par exemple). Elle est de type *string*. 
- FACILITY_DOM, FACILITY_SDOM, FACILITY_TYPE : elles indiquent la classification des types d'équipements à différents niveaux (catégories, sous-catégories et types). Ce sont des codes de types *string*. 
- BPE_MEASURE : elle est un indicateur d'intérêt de type *string* (A COMPLETER)
- OBS_VALUE : elle indique le nombre d'équipement décrit dans la ligne par les colonnes précédentes présents dans la zone géographique. Elle est de type *int*, mais panda l'a déjà repéré donc pas besoin de faire de modifications. 

Les autres colonnes concernent un degré de précision quant aux équipements que nous considérons trop élevé pour être pertinent (s'il existe une zone illuminée par exemple avec la variable LIGHTED), donc nous les laissons de côté. 

> **Remarque** TIME_PERIOD indique la période d'observation, lors de l'enquête. En l'occurrence, dans ce jeu de données, cette variable est inutile puisqu'elle contient une unique modalité : 2024. De même, FACILITY_DOM ne prend qu'une valeur : F, qui signifie "Sports, loisirs et cultures".

In [6]:
#vérification que TIME_PERIOD n'a qu'une valeur 
df_infrastructures["TIME_PERIOD"].value_counts()

TIME_PERIOD
2024    1351761
Name: count, dtype: int64

In [77]:
#vérification que FACILITY_DOM n'a qu'une valeur
df_infrastructures["FACILITY_DOM"].value_counts()

FACILITY_DOM
F    1351761
Name: count, dtype: int64

In [78]:
cols_kept = [
    "GEO", "GEO_OBJECT",
    "FACILITY_SDOM", "FACILITY_TYPE",
    "BPE_MEASURE", "OBS_VALUE"
]

df_infrastructures_typed = df_infrastructures[cols_kept].copy() #on fait une copie pour garder quelque part les données originales
df_infrastructures_typed["GEO"] = df_infrastructures_typed["GEO"].astype(str)
df_infrastructures_typed["GEO_OBJECT"] = df_infrastructures_typed["GEO_OBJECT"].astype(str)
df_infrastructures_typed["FACILITY_SDOM"] = df_infrastructures_typed["FACILITY_SDOM"].astype(str)
df_infrastructures_typed["FACILITY_TYPE"] = df_infrastructures_typed["FACILITY_TYPE"].astype(str)
df_infrastructures_typed["BPE_MEASURE"] = df_infrastructures_typed["BPE_MEASURE"].astype(str)

Etant donné la taille considérable du fichier (DS_BPE_SPORT_CULTURE_2024_data.csv), il vaut mieux le transformer au format Parquet pour éviter des temps d'exécution trop élevés. 

In [15]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [79]:
df_infrastructures_typed.to_parquet(
    "s3://clemencelucas/projet_python_vote_rn/data/intermediate/bpe_parquet_v1",
    engine = "pyarrow",
    index = False, 
    partition_cols = 'GEO_OBJECT'
)

df_infrastructures_typed.head(10)

,GEO,GEO_OBJECT,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,69,DEP,F1,F101,FACILITIES,1
1,12,DEP,F1,F101,FACILITIES,1
2,59,DEP,F1,F101,FACILITIES,1
3,18,DEP,F1,F101,FACILITIES,1
4,45,DEP,F1,F101,FACILITIES,1
5,86,DEP,F1,F101,FACILITIES,2
6,74,DEP,F1,F101,FACILITIES,1
7,89,DEP,F1,F101,FACILITIES,1
8,27,DEP,F1,F101,FACILITIES,1
9,57,DEP,F1,F101,FACILITIES,1


### ii. Métadonnées

Les métadonnées seront utiles pour avoir des résultats lisibles. Les informations présentes dans le document data comportent essentiellement des codes (F103 pour Tennis par exemple), probablement pour prendre moins d'espace de stockage. Les informations présentes dans le document metadata permettront de clarifier ces clés d'identification. 

In [148]:
path_infrastructures_meta_s3 = "s3://clemencelucas/projet_python_vote_rn/data/raw/DS_BPE_SPORT_CULTURE_2024_metadata.csv"
df_infrastructures_meta = pd.read_csv(path_infrastructures_meta_s3, sep=';') 

In [149]:
df_infrastructures_meta.head(20)

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,BPE_MEASURE,Mesure BPE,FACILITIES,Nombre d'équipements
1,BPE_MEASURE,Mesure BPE,PLAYGROUNDS,Nombre d'aires de pratiques sportives d'un mêm...
2,BPE_MEASURE,Mesure BPE,ROOMS,Nombre de salles de cinéma ou théâtre
3,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,1,Au-dessus de 1500 personnes
4,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,3,De 301 à 700 Personnes
5,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,2,De 701 à 1500 personnes
6,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,4,De 101 à 300 personnes
7,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,5,Inférieur ou égal à 100 personnes
8,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,_U,Indéterminé
9,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,_Z,Sans objet


In [150]:
df_infrastructures_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35722 entries, 0 to 35721
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   COD_VAR  35722 non-null  object
 1   LIB_VAR  35722 non-null  object
 2   COD_MOD  35722 non-null  object
 3   LIB_MOD  35722 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


Comme pour le document data, d'une part on n'a pas de valeurs nulles et d'autre part panda considère les 4 colonnes du fichier des métadonnées comme étant de type *object*, alors qu'elles sont toutes de type *string*. On les modifie donc, de la même manière en créant une copie du fichier qu'on modifie. 

In [151]:
df_infrastructures_meta_typed = df_infrastructures_meta.copy() #on fait une copie pour garder quelque part les données originales
df_infrastructures_meta_typed["COD_VAR"] = df_infrastructures_meta_typed["COD_VAR"].astype(str)
df_infrastructures_meta_typed["LIB_VAR"] = df_infrastructures_meta_typed["LIB_VAR"].astype(str)
df_infrastructures_meta_typed["COD_MOD"] = df_infrastructures_meta_typed["COD_MOD"].astype(str)
df_infrastructures_meta_typed["LIB_MOD"] = df_infrastructures_meta_typed["LIB_MOD"].astype(str)

Ce fichier est de taille très faible, donc on le laisse au format CSV.

# 2. Préparation des données

## a. Infrastructures sportives et culturelles

### i. Compréhension des différents niveaux géographiques

Dans un premier temps, il faut appréhender au mieux la base de données pour récupérer uniquement les informations utiles. En particulier, pour le plan géographique, on gardera uniquement ce qui concerne les communes. 

In [80]:
df_infrastructures_typed["GEO_OBJECT"].value_counts()

GEO_OBJECT
COM        352946
BV2022     179312
EPCI       168485
UU2020     128029
ARR        122801
ZE2020     118845
AAV2020    113880
DEP         85346
REG         45404
FRANCE      33512
ARM          3201
Name: count, dtype: int64

In [81]:
df_infrastructures_meta_typed["COD_VAR"].value_counts()


COD_VAR
GEO                            35630
FACILITY_TYPE                     37
ERP_CATEGORY                       7
SANITARY_ACCESSIBILITY             4
INDOOR                             4
LIGHTED                            4
LOCKER_ROOM_ACCESSIBILITY          4
MULTIPLEX_CINEMA                   4
FREE_ACCESS                        4
SHOWER                             4
SEASONAL_OPENING                   4
SANITARY                           4
PRACTICE_AREA_ACCESSIBILITY        4
BPE_MEASURE                        3
FACILITY_SDOM                      3
FACILITY_DOM                       1
TIME_PERIOD                        1
Name: count, dtype: int64

Les métadonnées ne renseignent rien sur la variable GEO_OBJECT, donc il faut chercher les informations ailleurs. Dans la [table d'appartenance géographique des communes au 1er janvier 2024](https://www.insee.fr/fr/information/7671844), on retrouve les notations DEP, REG, EPCI, ARR, ZE2020, UU2020, AAV2020, BV2022 et leur libellé : 
- AAV2020 : Aire d'Attraction des Villes en 2020
- ARR : Arrondissement 
- BV2022 : Bassin de Vie en 2022
- DEP : Département 
- EPCI : Intercommunalité - Métropole
- REG : Région
- UU2020 : Unité Urbaine en 2020
- ZE2020 : Zone d'emploi en 2020

L'Insee définit également les [EPCI](https://www.insee.fr/fr/information/2510634) comme des Etablissement Public de Coopération Intercommunal. 
Dans la description du [Code Officiel Géographique (COG)](https://www.insee.fr/fr/information/7766585), COM est libellé Code Commune et on retrouve le même résultat pour DEP, REG, ARR que précédemment. 

> Dans la [documentation](https://www.insee.fr/fr/metadonnees/source/operation/s2216/presentation) du dossier, l'Insee précise toutefois que *"dans les communes à arrondissements (Paris-Lyon-Marseille), l'arrondissement a été retenu comme niveau communal pour localiser les équipements."* 

Dans son [résumé du dossier](https://www.insee.fr/fr/metadonnees/source/serie/s1161), l'Insee précise que la base de données couvre le territoire métropolitain et les 5 départements d'outre-mer (Réunion, Guyane, Martinique, Guadeloupe, Mayotte). Néanmoins, il est également écrit que *"Depuis septembre 2018, des données concernant la BPE sont diffusées en évolution sur l'ensemble du territoire, ***hors Mayotte***.*
Etant donné la confusion possible concernant Mayotte, on choisira probablement de négliger ce territoire pour ce projet. Pour le reste, on peut donc considérer que FRANCE désigne la France entière, y compris les territoires d'outre-mer, en gardant en tête qu'il risque d'y avoir une petite variabilité à cause de la non-considération de Mayotte. 

Il reste à traiter le problème des ARM (a priori Arrondissements Municipaux). 

Pour la suite, afin d'avoir des temps d'exécution plus court, on crée des data frame pour chaque niveau géographique pertinent : 
- COM : la commune, puisque c'est notre unité statistique pour ce projet
- FRANCE : peut être pertinent pour comparer par rapport à des moyennes nationales par exemple
- DEP : le département peut-être plus lisible si l'on fait des cartes et peut être intéressant à regarder 
- ARM : l'idée étant de vérifier la nature de cette catégorie, et s'il s'agit bien des Arrondissements Municipaux, il faudra probablement agréger les résultats pour les mettre dans le data frame des communes. 

In [82]:
df_infrastructures_com = df_infrastructures_typed[ df_infrastructures_typed["GEO_OBJECT"] == 'COM'].copy()
df_infrastructures_com.head(5)

,GEO,GEO_OBJECT,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
2179,13027,COM,F1,F120,PLAYGROUNDS,9
2189,33550,COM,F1,F120,PLAYGROUNDS,7
2195,53262,COM,F1,F111,PLAYGROUNDS,2
2196,17097,COM,F1,F111,PLAYGROUNDS,1
2197,28214,COM,F1,F111,PLAYGROUNDS,4


In [83]:
df_infrastructures_arm = df_infrastructures_typed[ df_infrastructures_typed["GEO_OBJECT"] == 'ARM'].copy()
df_infrastructures_arm.head(5)

,GEO,GEO_OBJECT,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
3108,13213,ARM,F1,F106,PLAYGROUNDS,12
3466,13216,ARM,F1,F111,PLAYGROUNDS,9
3720,75103,ARM,F1,F114,PLAYGROUNDS,2
3744,75105,ARM,F1,F116,PLAYGROUNDS,5
3759,13213,ARM,F1,F121,PLAYGROUNDS,11


In [84]:
df_infrastructures_dep = df_infrastructures_typed[ df_infrastructures_typed["GEO_OBJECT"] == 'DEP'].copy()
df_infrastructures_dep.head(5)

,GEO,GEO_OBJECT,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,69,DEP,F1,F101,FACILITIES,1
1,12,DEP,F1,F101,FACILITIES,1
2,59,DEP,F1,F101,FACILITIES,1
3,18,DEP,F1,F101,FACILITIES,1
4,45,DEP,F1,F101,FACILITIES,1


In [85]:
df_infrastructures_france = df_infrastructures_typed[ df_infrastructures_typed["GEO_OBJECT"] == 'FRANCE'].copy()
df_infrastructures_france.head(5)

,GEO,GEO_OBJECT,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
2188,F,FRANCE,F1,F129,PLAYGROUNDS,11488
2360,F,FRANCE,F1,F111,PLAYGROUNDS,38639
2362,F,FRANCE,F1,F109,PLAYGROUNDS,2705
326244,F,FRANCE,F1,F103,PLAYGROUNDS,38895
340157,F,FRANCE,F1,F121,PLAYGROUNDS,18388


Comme on a supprimé les colonnes relatives à certaines précisions (présence d'équipement adaptés aux personnes handicapés, vestiaires, etc par exemple), on peut déjà agréger certains résultats. 

In [86]:
df_infrastructures_com_agg = (
    df_infrastructures_com
    .groupby(
        ["GEO", "FACILITY_SDOM", "FACILITY_TYPE", "BPE_MEASURE"],
        as_index = False
    )["OBS_VALUE"]
    .sum()
)

df_infrastructures_com_agg.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,01001,F1,F102,FACILITIES,1
1,01001,F1,F102,PLAYGROUNDS,2
2,01001,F1,F111,FACILITIES,1
3,01001,F1,F111,PLAYGROUNDS,1
4,01001,F1,F113,FACILITIES,1


In [87]:
df_infrastructures_arm_agg = (
    df_infrastructures_arm
    .groupby(
        ["GEO", "FACILITY_SDOM", "FACILITY_TYPE", "BPE_MEASURE"],
        as_index = False
    )["OBS_VALUE"]
    .sum()
)

df_infrastructures_arm_agg.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,13201,F1,F101,FACILITIES,1
1,13201,F1,F101,PLAYGROUNDS,1
2,13201,F1,F107,FACILITIES,2
3,13201,F1,F107,PLAYGROUNDS,2
4,13201,F1,F111,FACILITIES,2


In [88]:
df_infrastructures_dep_agg = (
    df_infrastructures_dep
    .groupby(
        ["GEO", "FACILITY_SDOM", "FACILITY_TYPE", "BPE_MEASURE"],
        as_index = False
    )["OBS_VALUE"]
    .sum()
)

df_infrastructures_dep_agg.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,01,F1,F101,FACILITIES,33
1,01,F1,F101,PLAYGROUNDS,61
2,01,F1,F102,FACILITIES,385
3,01,F1,F102,PLAYGROUNDS,507
4,01,F1,F103,FACILITIES,201


In [89]:
df_infrastructures_france_agg = (
    df_infrastructures_france
    .groupby(
        ["GEO", "FACILITY_SDOM", "FACILITY_TYPE", "BPE_MEASURE"],
        as_index = False
    )["OBS_VALUE"]
    .sum()
)

df_infrastructures_france_agg.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,F,F1,F101,FACILITIES,3636
1,F,F1,F101,PLAYGROUNDS,6151
2,F,F1,F102,FACILITIES,21532
3,F,F1,F102,PLAYGROUNDS,28590
4,F,F1,F103,FACILITIES,15441


### ii. Traitement des ARM

Pour déterminer à quoi correspond la modalité ARM, on regarde les valeurs prises par la variable GEO. 

In [90]:
df_infrastructures_arm_agg["GEO"].unique()

array(['13201', '13202', '13203', '13204', '13205', '13206', '13207',
       '13208', '13209', '13210', '13211', '13212', '13213', '13214',
       '13215', '13216', '69381', '69382', '69383', '69384', '69385',
       '69386', '69387', '69388', '69389', '75101', '75102', '75103',
       '75104', '75105', '75106', '75107', '75108', '75109', '75110',
       '75111', '75112', '75113', '75114', '75115', '75116', '75117',
       '75118', '75119', '75120'], dtype=object)

In [91]:
len(df_infrastructures_arm_agg["GEO"].unique())

45

On a bien les 20+16+9 = 45 arrondissements de Paris, Marseille et Lyon, et on retrouve dans le [COG](https://www.insee.fr/fr/information/7766585) les codes utilisés par l'Insee pour désigner ces-dits arrondissements. 
> Dans le fichier *Liste des communes* avec le filtre TYPECOM = ARM. 

On peut maintenant agréger les résultats des arrondissements municipaux par commune, dans l'objectif de comparer avec ce qui est indiqué pour Paris, Marseille et Lyon dans le dataframe commune (df_infrastructures_com_agg). Pour cela, on crée une nouvelle colonne GEO_COM qui comporte une clé indiquant la commune (Paris, Marseille ou Lille) de l'arrondissement. On prend le premier chiffre de GEO (1 pour Marseille, 6 pour Lyon et 7 pour Paris) car cela suffit pour l'identification. 

In [92]:
df_infrastructures_arm_agg["GEO_COM"] = df_infrastructures_arm_agg["GEO"].str[0]
df_infrastructures_arm_agg.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE,GEO_COM
0,13201,F1,F101,FACILITIES,1,1
1,13201,F1,F101,PLAYGROUNDS,1,1
2,13201,F1,F107,FACILITIES,2,1
3,13201,F1,F107,PLAYGROUNDS,2,1
4,13201,F1,F111,FACILITIES,2,1


Maintenant on peut agréger avec FACILITY_DOM, FACILITY_SDOM, FACILITY_TYPE, BPE_MEASURE et GEO_COM, qu'on replace en premier pour plus de lisibilité. 

In [93]:
df_infrastructures_arm_agg = (
    df_infrastructures_arm_agg
    .groupby(
        [
            "GEO_COM",
            "FACILITY_SDOM",
            "FACILITY_TYPE",
            "BPE_MEASURE",
        ],
        as_index=False
    )["OBS_VALUE"]
    .sum()
)

df_infrastructures_arm_agg.head(5)

,GEO_COM,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,1,F1,F101,FACILITIES,30
1,1,F1,F101,PLAYGROUNDS,34
2,1,F1,F102,FACILITIES,60
3,1,F1,F102,PLAYGROUNDS,76
4,1,F1,F103,FACILITIES,36


On remplace GEO_COM par le vrai code COG des communes : 
- Marseille : 13055
- Lyon : 69123
- Paris : 75056

In [94]:
vrai_cog = {
    "1" : "13055", #Marseille
    "6" : "69123", #Lyon
    "7" : "75056" #Paris
}

df_infrastructures_arm_agg["GEO_COM"] = (
    df_infrastructures_arm_agg["GEO_COM"]
    .map(vrai_cog)
)

df_infrastructures_arm_MLP = df_infrastructures_arm_agg.rename(columns = {"GEO_COM" : "GEO"}).copy()

df_infrastructures_arm_MLP.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
0,13055,F1,F101,FACILITIES,30
1,13055,F1,F101,PLAYGROUNDS,34
2,13055,F1,F102,FACILITIES,60
3,13055,F1,F102,PLAYGROUNDS,76
4,13055,F1,F103,FACILITIES,36


Maintenant on veut comparer ce dernier dataframe avec le contenu de celui des communes (df_infrastructures_com_agg) pour les villes de Marseille, Lyon et Paris uniquement. 

In [95]:
MLP_cog = ["13055", "69123", "75056"]

df_infrastructures_com_MLP = (
    df_infrastructures_com_agg
    .query("GEO in @MLP_cog")
    .copy()
)

df_infrastructures_com_MLP.head(5)

,GEO,FACILITY_SDOM,FACILITY_TYPE,BPE_MEASURE,OBS_VALUE
28405,13055,F1,F101,FACILITIES,30
28406,13055,F1,F101,PLAYGROUNDS,34
28407,13055,F1,F102,FACILITIES,60
28408,13055,F1,F102,PLAYGROUNDS,76
28409,13055,F1,F103,FACILITIES,36


On remarque déjà grâce aux aperçus des tableaux qu'ils ne commencent pas au même index, donc on commence par réindexer les tableaux avant de les comparer.

In [96]:
cols = ["GEO","FACILITY_SDOM","FACILITY_TYPE","BPE_MEASURE","OBS_VALUE"]

arm = df_infrastructures_arm_MLP[cols].sort_values(cols).reset_index(drop=True)
com = df_infrastructures_com_MLP[cols].sort_values(cols).reset_index(drop=True)

arm.equals(com)

True

On confirme ainsi que les données présentes dans le dataframe des communes correspondent aux données agrégées du dataframe des arrondissements municipaux. On peut donc se délester de ce niveau géographique et se concentrer sur celui des communes (sans avoir à modifier df_infrastructures_com_agg). 

### iii. Données manquantes

On regarde l'ensemble des modalités des variables des dataframes étudiés.

In [132]:
df_infrastructures_com_agg["GEO"].value_counts(dropna=False)

GEO
72181    61
42218    59
85194    59
11262    58
31555    58
         ..
88241     1
88249     1
88256     1
88260     1
88272     1
Name: count, Length: 29889, dtype: int64

In [133]:
modalites_geo = (
    df_infrastructures_com_agg["GEO"]
    .unique()
    .tolist()
)

libelles_geo = (
    df_infrastructures_meta_typed
    .loc[
        (df_infrastructures_meta_typed["COD_VAR"] == "GEO") &
        (df_infrastructures_meta_typed["COD_MOD"].isin(modalites_geo)),
        ["COD_MOD", "LIB_MOD"]
    ]
    .drop_duplicates(subset=["COD_MOD"])
    .sort_values("COD_MOD")
)

libelles_geo

,COD_MOD,LIB_MOD
6985,01001,L'Abergement-Clémenciat
6986,01002,L'Abergement-de-Varey
4837,01004,Ambérieu-en-Bugey
6988,01005,Ambérieux-en-Dombes
6990,01006,Ambléon
...,...,...
35714,97613,M'Tsangamouji
35715,97614,Ouangani
35716,97615,Pamandzi
35717,97616,Sada


> **Remarque** : On a rajouté le drop_duplicate car on avait vu dans l'aperçu qu'au moins une commune était présente deux fois (mêmes COD_MOD et LIB_MOD). 

On observe une différence de 1180 communes entre les deux résultats. On cherche donc à savoir combien de communes présentes dans les données n'ont pas de libellés associés dans les métadonnées. Pour les autres, il est probable qu'il s'agisse de petites communes (villages de moins de quelques centaines d'habitants par exemple) qui n'ont tout simplement pas d'équipement des types étudiés. Ainsi, il n'est pas nécessaire de regarder les communes qui pourraient être présentes dans les métadonnées et pas dans les données. 

In [134]:
com_data = (
    df_infrastructures_com_agg["GEO"]
    .str.strip()
    .unique()
)

com_meta = (
    df_infrastructures_meta_typed
    .loc[df_infrastructures_meta_typed["COD_VAR"] == "GEO", "COD_MOD"]
    .str.strip()
    .unique()
)

com_data_set = set(com_data)
com_meta_set = set(com_meta)

com_manquantes = sorted(com_data_set - com_meta_set)
len(com_manquantes), com_manquantes

(1180,
 ['1224',
  '1228',
  '1229',
  '1232',
  '1235',
  '1238',
  '1245',
  '1246',
  '1247',
  '1249',
  '1250',
  '1254',
  '1258',
  '1262',
  '1264',
  '1265',
  '1266',
  '1269',
  '1272',
  '1273',
  '1283',
  '1286',
  '4242',
  '4244',
  '4245',
  '5001',
  '5003',
  '5004',
  '5006',
  '5007',
  '5008',
  '5009',
  '5010',
  '5011',
  '5012',
  '5013',
  '5014',
  '5016',
  '5017',
  '5018',
  '5019',
  '5021',
  '5022',
  '5023',
  '5024',
  '5025',
  '5026',
  '5027',
  '5028',
  '5029',
  '5031',
  '5032',
  '5033',
  '5035',
  '5036',
  '5037',
  '5038',
  '5039',
  '5040',
  '5044',
  '5045',
  '5046',
  '5047',
  '5048',
  '5049',
  '5050',
  '5051',
  '5052',
  '5053',
  '5054',
  '5055',
  '5056',
  '5057',
  '5058',
  '5059',
  '5061',
  '5062',
  '5063',
  '5064',
  '5065',
  '5066',
  '5068',
  '5070',
  '5071',
  '5072',
  '5073',
  '5074',
  '5075',
  '5076',
  '5077',
  '5078',
  '5079',
  '5081',
  '5084',
  '5085',
  '5086',
  '5087',
  '5089',
  '5090',
  '

On remarque que tous ces codes ne sont composés que de 4 chiffres au lieu des 5 habituels. Il parait raisonnable de penser qu'il manque un zéro au début de la chaîne de caractère. 

In [135]:
df_infrastructures_com_agg["GEO_NORM"] = (
    df_infrastructures_com_agg["GEO"]
    .str.zfill(5)
)

df_infrastructures_meta_typed["COD_MOD_NORM"] = (
    df_infrastructures_meta_typed["COD_MOD"]
    .str.zfill(5)
)

On a rajouté des 0 pour être sûr que les codes fassent 5 caractères. On peut revérifier à nouveau chercher les valeurs manquantes et voir combien il en reste. 

In [136]:
com_data_norm = (
    df_infrastructures_com_agg["GEO_NORM"]
    .str.strip()
    .unique()
)

com_meta_norm = (
    df_infrastructures_meta_typed
    .loc[df_infrastructures_meta_typed["COD_VAR"] == "GEO", "COD_MOD_NORM"]
    .str.strip()
    .unique()
)

com_data_norm_set = set(com_data_norm)
com_meta_norm_set = set(com_meta_norm)

com_manquantes_norm = sorted(com_data_norm_set - com_meta_norm_set)
len(com_manquantes_norm), com_manquantes_norm

(0, [])

On a rajouté des 0 pour être sûr que les codes fassent 5 caractères. On peut revérifier à nouveau chercher les valeurs manquantes et voir combien il en reste. 

Le problème est résolu, on a bien toutes les communes. On considère donc pour la suite que les communes qui ne sont pas mentionnées dans les données n'ont pas d'équipements des types étudiés. 

In [137]:
df_infrastructures_com_agg["FACILITY_SDOM"].value_counts(dropna=False)

FACILITY_SDOM
F1    240564
F2     35262
F3     19947
Name: count, dtype: int64

In [138]:
modalites_facility_sdom = (
    df_infrastructures_com_agg["FACILITY_SDOM"]
    .unique()
    .tolist()
)

libelles_facility_sdom = (
    df_infrastructures_meta_typed
    .loc[
        (df_infrastructures_meta_typed["COD_VAR"] == "FACILITY_SDOM") &
        (df_infrastructures_meta_typed["COD_MOD"].isin(modalites_facility_sdom)),
        ["COD_MOD", "LIB_MOD"]
    ]
    .sort_values("COD_MOD")
)

libelles_facility_sdom

,COD_MOD,LIB_MOD
11,F1,Equipements sportifs
12,F2,Equipements de loisirs
13,F3,Equipements culturels et socioculturels


Les équipements des communes sont réparties en trois sous-catégories : 
- F1 : équipements sportifs
- F2 : équipements de loisirs
- F3 : équipements culturels et socio-culturels

Il n'y a pas de données manquantes dans cette colonne. 

In [139]:
df_infrastructures_com_agg["FACILITY_TYPE"].value_counts(dropna=False)

FACILITY_TYPE
F113    34253
F102    30440
F111    27900
F103    24933
F203    23230
F116    22672
F121    14264
F307    14179
F204     9291
F106     9241
F114     8533
F129     8006
F107     7515
F120     6817
F130     6242
F101     5649
F124     5032
F109     4632
F123     4344
F127     3819
F118     3664
F128     3048
F125     2872
F122     2428
F303     1711
F201     1701
F108     1577
F315     1358
F312     1154
F126     1064
F202     1040
F105      763
F314      661
F119      550
F313      491
F305      393
F110      306
Name: count, dtype: int64

In [140]:
modalites_facility_type = (
    df_infrastructures_com_agg["FACILITY_TYPE"]
    .unique()
    .tolist()
)

libelles_facility_type = (
    df_infrastructures_meta_typed
    .loc[
        (df_infrastructures_meta_typed["COD_VAR"] == "FACILITY_TYPE") &
        (df_infrastructures_meta_typed["COD_MOD"].isin(modalites_facility_type)),
        ["COD_MOD", "LIB_MOD"]
    ]
    .sort_values("COD_MOD")
)

libelles_facility_type

,COD_MOD,LIB_MOD
14,F101,Bassin de natation
15,F102,Boulodrome
16,F103,Tennis
17,F105,Domaine skiable
18,F106,Centre équestre
19,F107,Athlétisme
20,F108,Terrain de golf
21,F109,Parcours sportif/santé
22,F110,Sports de glace
23,F111,Plateaux et terrains de jeux extérieurs


Les équipements des communes sont répartis en 37 types et il n'y a pas de données manquantes. 

In [141]:
df_infrastructures_com_agg["BPE_MEASURE"].value_counts(dropna=False)

BPE_MEASURE
FACILITIES     157382
PLAYGROUNDS    137748
ROOMS             643
Name: count, dtype: int64

In [142]:
modalites_bpe_measure = (
    df_infrastructures_com_agg["BPE_MEASURE"]
    .unique()
    .tolist()
)

libelles_bpe_measure = (
    df_infrastructures_meta_typed
    .loc[
        (df_infrastructures_meta_typed["COD_VAR"] == "BPE_MEASURE") &
        (df_infrastructures_meta_typed["COD_MOD"].isin(modalites_bpe_measure)),
        ["COD_MOD", "LIB_MOD"]
    ]
    .sort_values("COD_MOD")
)

libelles_bpe_measure

,COD_MOD,LIB_MOD
0,FACILITIES,Nombre d'équipements
1,PLAYGROUNDS,Nombre d'aires de pratiques sportives d'un mêm...
2,ROOMS,Nombre de salles de cinéma ou théâtre


La mesure BPE est un indicateur d'intérêt réparti en 3 catégories. Il n'apporte pas grand chose mais permet tout de même de mieux comprendre le sens de OBS_VALUE, donc nous l'avons gardé. Il n'y a pas de valeurs manquantes. 

In [143]:
df_infrastructures_com_agg["OBS_VALUE"].isna().sum()

np.int64(0)

In [144]:
(df_infrastructures_com_agg["OBS_VALUE"] == 0).any()

np.False_

La variable OBS_VALUE est de type *int* donc on voit qu'elle n'a pas de valeurs manquantes. Cependant, elle n'a pas de 0 donc aucune ligne n'indique qu'une commune **n'a pas** tel ou tel équipement. On connaît seulement les équipements que possède une commune, pas ceux qu'elle ne possède pas. 

**Il n’y a pas de cases vides dans le jeu de données. Cependant, il faut garder en tête que l’absence d’équipement n’est pas documentée (il n’y a pas de OBS_VALUE à 0), donc il faudra probablement créer d’autres dataframes selon les graphiques que l’on veut faire. Par exemple, pour représenter sur une carte, il faudra probablement écrire des lignes pour les communes manquantes.**

## b. Elections présidentielles

Afin d'étudier le vote RN dans les communes françaises, nous avons choisi de nous concentrer sur les résultats à la présidentielle 2022 du premier tour, le 10 avril 2022. Nous avions initialement pensé à étudier les résultats au premier tour des législatives 2024, plus récents, mais avons réalisé que bon nombre de circonscriptions n'avaient pas de candidats RN, ce qui aurait pû créer un biais. Avec les présidentielles, nous avions l'assurance d'obtenir des résultats pour chaque commune. 

Entre le premier et le second tour de la présidentielle, nous avons naturellement porté notre attention sur les résultats du premier tour, puisqu'il y avait plus de candidats. 

> Remarque : peut-être détailler un peu plus

Les résultats des élections en France sont disponibles sur **[data.gouv.fr](https://www.data.gouv.fr/datasets/election-presidentielle-des-10-et-24-avril-2022-resultats-definitifs-du-1er-tour/)**. 
Malheureusement, le niveau communal n'est pas proposé, donc nous avons choisi de télécharger le niveau **sous-communal** dans l'optique d'agréger les résultats par la suite. 

In [ ]:
url_elect = "https://www.data.gouv.fr/api/1/datasets/r/6d9b33e5-667d-4c3e-9a0b-5fdf5baac708"
path_elect = "/home/onyxia/work/Projet-Python-S3/code/results_elect.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_elect)
response.raise_for_status()

with open(path_elect, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_extrait = pd.read_excel(path_elect, sheet_name=0)

print("Le dataframe df_extrait peut être traité comme un data frame classique avec pandas")

print("Informations sur le dataframe df_extrait qui est notre df initial")
print(df_extrait.columns)
print(len(df_extrait))


# la colonne unamed 53 correspond au nombre de voix de JLF, la colonne unamed 67 correspond à celui de marine le pen
liste_var = ['Code du département', 
    'Libellé du département', 
    'Code de la commune', 
    'Libellé de la commune',
    'Inscrits', 
    'Abstentions',
    '% Abs/Ins',
    'Unnamed: 53', 
    'Unnamed: 67']
df_extrait2 = df_extrait[liste_var]
df_extrait3 = df_extrait2.rename(columns={'Unnamed: 53': 'score_rn'})
df_elect = df_extrait3.rename(columns={'Unnamed: 67': 'score_lfi'})
print("Informations sur le dataframe df_elect")
print(df_elect.columns)
print(len(df_elect))
print(df_elect.head(4))

2) Préparation des données infrastructures
Dans un second temps, nous devons préparer le dataframe des infrastructures. Nous allons le passer d'un format long à un format large, autrement dit, au lieu d'avoir une ligne par infrastructure, nous aurons une ligne par ville et une colonne par infrastructure.

In [ ]:
print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())

In [ ]:
df_infra_large = (
    df_infrastruct
    .query("TIME_PERIOD == 2024")
    .groupby(["GEO", "FACILITY_TYPE"], as_index=False)["OBS_VALUE"]
    .sum()
    .pivot(index="GEO", columns="FACILITY_TYPE", values="OBS_VALUE")
    .fillna(0)
    .reset_index()
)


In [ ]:
print("Vérifications sur le dataframe df_infra_large")
print(df_infra_large.columns)
print(df_infra_large.size)
print(df_infra_large.head())

# 2. Statistiques descriptives 

Cette partie a pour but de vérifier que les bases sont propres et utilisables correctement. Les statistiques descriptives les statistiques descriptives permettront d'être sûres qu'il n'y a pas de données en double, de données manquantes ou de données aberrantes. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

## a. Infrastructures sportives et culturelles

# a. Elections présidentielles

Nous allons représenter les résultats aux élections présidentielles des deux partis choisis. Pour ce faire, dans un premier temps, il faut calculer 

In [ ]:
print("Le score moyen par commune du RN était")
print(df_elect["score_rn"].mean())
print(df_elect["score_rn"].median())
print("Le score moyen par commune de LFI était")
print(df_elect["score_lfi"].mean())
print(df_elect["score_lfi"].median())

Si on compare aux scores nationaux, on obtient des chiffres assez proches (Mme Le Pen ayant obtenu 23,15 % des voix au premier tour). La différence entre les deux chiffres peut être due au fait que nous avons fait une moyenne par commune sans pondérer par le nombre d'habitants, ce qui fait que les petites communes, plutôt en campagne et qui votent plus largement pour le RN sur sur-représentées. 

In [ ]:
# Préparation du code département : il faut le formater pour qu'il corresponde aux données des fonds de carte

df_elect['Code du département'] = (
    df_elect['Code du département']
    .astype(str)
    .str.zfill(2)
)

# Agrégation par département

# Moyenne des scores communaux

df_dep = (
    df_elect
    .groupby(['Code du département', 'Libellé du département'], as_index=False)
    .agg({
        'score_rn': 'mean',
        'score_lfi': 'mean'
    })
)


# Chargement du fond de carte départements

gdf_dep = gpd.read_file(
    "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
)

gdf_dep['code'] = gdf_dep['code'].astype(str).str.zfill(2)

# Fusion de la carte avec les données

carte_dep = gdf_dep.merge(
    df_dep,
    left_on='code',
    right_on='Code du département',
    how='left'
)

# Carte RN par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_rn',
    cmap='Blues',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score Mme Le Pen par département (1er tour 2022)")
ax.axis('off')
plt.show()

# Carte LFI par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_lfi',
    cmap='Reds',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score M. Mélenchon par département (1er tour 2022)")
ax.axis('off')
plt.show()


Les cartes suivantes sont cohérentes avec les statistiques publiées à propos des élections. Cependant, nous rappelons qu'ici, notre variable d'intérêt est la commune, c'est pour cela que nous n'avons pas pondéré par le nombre d'habitants donc les résultats publiés en ligne. Toutefois, nous ne voyons pas d'incohérence majeure.